# CC-cost for Sep/22

This script runs a `sql` code to create a panel data, based on the `case_records_current` table, where we have each `ICD-10` member change of status, and the link between the status change with change in daly weights. 
The calculation involves a lot of cleaning,   such as desconsidering impetous changes of status or *cancelled* icds status as the first status input, etc, we leave that for the sql code, and here is just the script to 
run it and export to the redshift.

Steps:

    - run this notebook to upload to Redshift;
    
    - use the new panel to make calculations, new tables, dashes...
***

## 0. Setup

In [1]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [2]:
df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/Care Cost Setembro 2022 - Sheet1.csv')
df.head(10)

,grupo_etario,Disease.Code.Value,care_cost_naveg,care_cost_enf_acolhimento,care_cost_enf_clinica,care_cost_enf_gestora,care_cost_enf_on_site,care_cost_Med_AA,care_cost_Med_Gestora,care_cost_Med_On_Site,...,care_cost_exames,care_cost_consultas,care_cost_terapias,care_cost_inter_proced,care_cost_PS,volumetria,care_cost_medio,care_cost_total,subcategoria,categoria_Alice
0,18-60,S423,$34.82,$75.32,$34.44,$0.00,$0.00,$46.19,$0.00,$0.00,...,$6.81,$0.00,$0.00,"$190,167.64",$380.00,1,"$190,745.22","$190,745.22",Fratura da diáfise do úmero,Fratura do ombro e do braço
1,18-60,F41,$4.89,$12.24,$8.89,$5.97,$4.27,$10.18,$45.20,$2.31,...,$102.30,$91.45,$5.06,$13.91,$11.30,412,$377.03,"$155,337.83",Outros transtornos ansiosos,Transtornos Ansiosos
2,0-2,P599,$0.00,$3.21,$37.87,$91.00,$0.00,$6.65,$338.77,$110.16,...,$0.00,$0.00,$0.00,"$21,398.89",$108.13,6,"$22,105.57","$132,633.44",Icterícia neonatal não especificada,Icterícia neonatal devida a outras causas e às...
3,18-60,F411,$2.81,$5.86,$8.86,$4.21,$1.36,$9.90,$36.32,$0.00,...,$7.31,$35.74,$0.00,$0.00,$2.47,834,$147.85,"$123,302.86",Ansiedade generalizada,Transtornos Ansiosos
4,18-60,N80,$3.31,$4.32,$5.10,$5.64,$0.00,$3.57,$14.75,$0.00,...,$26.91,$145.31,$0.00,$754.43,$0.00,106,$966.83,"$102,483.50",Endometriose,Endometriose
5,18-60,F412,$4.21,$8.15,$8.91,$8.93,$0.66,$9.77,$56.46,$0.00,...,$10.81,$41.09,$0.00,$92.86,$10.99,300,$309.24,"$92,772.59",Transtorno misto ansioso e depressivo,Transtornos Ansiosos
6,18-60,K21,$0.18,$2.32,$1.34,$7.63,$0.00,$1.64,$42.28,$21.98,...,$2.33,$72.10,$0.00,$566.88,$0.00,125,$726.45,"$90,806.48",Doença de refluxo gastroesofágico,"Doenças do esôfago, do estômago e do duodeno"
7,18-60,E66,$1.99,$4.39,$3.99,$2.59,$1.67,$3.46,$14.00,$0.00,...,$0.00,$42.55,$0.00,$23.07,$0.00,725,$112.66,"$81,681.14",Obesidade,Obesidade
8,18-60,F32,$1.80,$5.69,$0.53,$8.97,$0.00,$3.96,$35.63,$1.24,...,$29.57,$71.78,$0.00,$0.00,$22.18,307,$227.45,"$69,826.07",Episódios depressivos,Transtornos Depressivos
9,18-60,R10,$2.06,$9.51,$31.86,$9.19,$4.84,$17.18,$96.56,$49.84,...,$22.29,$46.24,$17.84,"$2,033.48",$83.69,28,"$2,480.78","$69,461.76",Dor abdominal e pélvica,Sinais e sintomas


In [4]:
df.rename(columns = {'Disease.Code.Value': 'disease_code_value'}, inplace = True)
df2= df[['grupo_etario', 'disease_code_value','volumetria', 'care_cost_medio', 'subcategoria',	'categoria_Alice']]

## 3. Export

In [5]:
di.redshift.insert_table(df2, table_name='care_cost_for_dalys_calculation', if_exists='replace')

Table care_cost_for_dalys_calculation inserted successfully, with 8188 rows.
